# Import

In [ ]:
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
from mesa.visualization.modules import CanvasGrid, ChartModule
from mesa.visualization.ModularVisualization import ModularServer

import warnings
import random
import math
import numpy as np

# Init 

## Basic model

In [ ]:
GLOBAL_METABOLISM = 8
GLOBAL_VISION = 2
GLOBAL_MAX_SUGAR = 8
GLOBAL_INIT_AGENT_SUGAR = 20

# 定义SugarAgent类
class SugarAgent(Agent):
    def __init__(self, unique_id, model, pos, sugar):
        super().__init__(unique_id, model)  # 显式调用父类的初始化方法
        self.pos = pos
        self.sugar = sugar
        self.metabolism = GLOBAL_METABOLISM
        self.vision = GLOBAL_VISION
    
    def step(self):
        self.move()
        self.eat()
        self.metabolize()
        
    def move(self):
        neighborhood = self.model.grid.get_neighborhood(self.pos, moore=True, radius=self.vision)
        # 只考虑有糖的位置
        valid_cells = [cell for cell in neighborhood if any(isinstance(obj, SugarPatch) for obj in self.model.grid.get_cell_list_contents(cell))]

        if not valid_cells:
            return  # No available moves, stay in place

        # 找到最大糖量
        max_sugar = max(max(obj.sugar for obj in self.model.grid.get_cell_list_contents(pos) if isinstance(obj, SugarPatch)) for pos in valid_cells)

        # 找出所有糖量等于最大糖量的单元格
        max_sugar_cells = [pos for pos in valid_cells if any(obj.sugar == max_sugar for obj in self.model.grid.get_cell_list_contents(pos) if isinstance(obj, SugarPatch))]

        # 随机选择一个最大糖量的单元格
        new_position = self.random.choice(max_sugar_cells)

        self.model.grid.move_agent(self, new_position)

    def eat(self):
        patch = self.model.grid.get_cell_list_contents([self.pos])[0]
        if isinstance(patch, SugarPatch):
            self.sugar += patch.sugar
            patch.sugar = 0
    
    def metabolize(self):
        self.sugar -= self.metabolism
        if self.sugar <= 0:
            self.model.grid.remove_agent(self)
            self.model.schedule.remove(self)

# 定义SugarPatch类
class SugarPatch(Agent):
    def __init__(self, unique_id, model, pos, max_sugar):
        super().__init__(unique_id, model)  # 显式调用父类的初始化方法
        self.pos = pos
        self.max_sugar = max_sugar
        self.sugar = random.randint(0,max_sugar)
    
    def step(self):
        self.regrow()
        
    def regrow(self):
        if self.sugar < self.max_sugar:
            self.sugar += random.randint(0,1)

# 定义Sugarscape模型
class Sugarscape(Model):
    def __init__(self, width, height, initial_agents):
        super().__init__()
        self.grid = MultiGrid(width, height, torus=False) # torus means circle
        # Initialize the current_id property to ensure that the next_id method can work properly
        self.current_id = 0

        # Randomly shuffle the order of agents and activate each agent once.
        self.schedule = RandomActivation(self)
        """
        def step(self):
        self.model.random.shuffle(self.agents)
        for agent in self.agents:
            agent.step()
        """

        # Collect data at model level and agent level, for now it just use RandomActivation to collect the 
        # number of agents
        self.datacollector = DataCollector(
        {
            "AgentCount": lambda m: sum(1 for a in m.schedule.agents if isinstance(a, SugarAgent))
        }
        )

        # 初始化糖资源
        # self.grid.coord_iter() is a method provided by MultiGrid that is used to iterate over each 
        # cell in the grid.
        # contents: represents the contents of the cell
        for (contents, pos) in self.grid.coord_iter():
            max_sugar = GLOBAL_MAX_SUGAR
            patch = SugarPatch(self.next_id(), self, pos, max_sugar)
            # Suppress specific warnings
            with warnings.catch_warnings():
                warnings.simplefilter("ignore", UserWarning)
                self.grid.place_agent(patch, pos)
            self.schedule.add(patch)
            

        # 初始化代理
        available_positions = [(x, y) for x in range(self.grid.width) for y in range(self.grid.height)]
        self.random.shuffle(available_positions)

        for i in range(initial_agents):
            if not available_positions:
                raise ValueError("No available positions to place new agents.")

            pos = available_positions.pop()
            sugar = GLOBAL_INIT_AGENT_SUGAR
            agent = SugarAgent(self.next_id(), self, pos, sugar)
            # Suppress specific warnings
            with warnings.catch_warnings():
                warnings.simplefilter("ignore", UserWarning)
                self.grid.place_agent(agent, pos)
            self.schedule.add(agent)
        
    def step(self):
        self.schedule.step()
        self.datacollector.collect(self)
"""
# local运行模型
model = Sugarscape(50, 50, 100)
for i in range(15):
    model.step()

# 收集数据
data = model.datacollector.get_model_vars_dataframe()
print(data)
"""
# Browser-based visual interface
def agent_portrayal(agent):
    if isinstance(agent, SugarAgent):
        portrayal = {
            "Shape": "circle",
            "Filled": "true",
            "r": 0.5,
            "Color": "blue",
            "Layer": 1,
            "text": agent.sugar,
            "text_color": "white",
        }
    elif isinstance(agent, SugarPatch):
        portrayal = {
            "Shape": "rect",
            "w": 1,
            "h": 1,
            "Filled": "true",
            "Color": "green",
            "Layer": 0,
            "text": agent.sugar,
            "text_color": "black",
        }
    return portrayal

grid = CanvasGrid(agent_portrayal, 50, 50, 1500, 1500)
chart = ChartModule([{"Label": "AgentCount", "Color": "Black"}])



## Run

In [ ]:
server = ModularServer(Sugarscape,
                       [grid, chart],
                       "Sugarscape Model",
                       {"width": 50, "height": 50, "initial_agents": 200})

server.port = 8538
server.launch()

# Update model

In [ ]:
GLOBAL_METABOLISM = 3
GLOBAL_REGROW_RATE = 1
GLOBAL_VISION = 3
GLOBAL_MAX_SUGAR = 5
GLOBAL_MAX_MONEY = 5
GLOBAL_INIT_AGENT_SUGAR = 20
GLOBAL_INIT_AGENT_MONEY = 20
GLOBAL_THRESHOLD = 10

In [ ]:
# 全局变量 Global variables
GLOBAL_METABOLISM = 3
GLOBAL_REGROW_RATE = 1
GLOBAL_VISION = 3
GLOBAL_MAX_SUGAR = 5
GLOBAL_MAX_MONEY = 5
GLOBAL_INIT_AGENT_SUGAR = 20
GLOBAL_INIT_AGENT_MONEY = 20
GLOBAL_THRESHOLD = 10

# 定义买家代理类 Define Buyer Agent Class
class Buyer(Agent):
    def __init__(self, unique_id, model, pos, money, initial_sugar):
        super().__init__(unique_id, model)
        self.pos = pos
        self.money = money
        self.sugar = initial_sugar
        self.metabolism = GLOBAL_METABOLISM
        self.vision = GLOBAL_VISION
        self.price = 1  # 初始价格 Initial price

    def step(self):
        self.move()
        self.collect_money()
        self.set_price()
        self.trade()
        self.metabolize()
        
    def move(self):
        neighborhood = self.model.grid.get_neighborhood(self.pos, moore=True, radius=self.vision)
        valid_cells = [cell for cell in neighborhood if any(isinstance(obj, MoneyPatch) for obj in self.model.grid.get_cell_list_contents(cell))]

        if not valid_cells:
            return  # No available moves, stay in place

        max_money = max(max(obj.money for obj in self.model.grid.get_cell_list_contents(pos) if isinstance(obj, MoneyPatch)) for pos in valid_cells)
        max_money_cells = [pos for pos in valid_cells if any(obj.money == max_money for obj in self.model.grid.get_cell_list_contents(pos) if isinstance(obj, MoneyPatch))]
        new_position = self.random.choice(max_money_cells)

        self.model.grid.move_agent(self, new_position)

    def collect_money(self):
        patch = next((obj for obj in self.model.grid.get_cell_list_contents([self.pos]) if isinstance(obj, MoneyPatch)), None)
        if patch:
            self.money += patch.money
            patch.money = 0

    def set_price(self):
        if self.sugar > 2 * GLOBAL_THRESHOLD:
            self.price = 1 / 2
        elif self.sugar > GLOBAL_THRESHOLD:
            self.price = 1
        else:
            self.price = 2 

    def trade(self):
        # 获取视野范围内的所有位置  Get all positions within the field of view
        neighborhood = self.model.grid.get_neighborhood(self.pos, moore=True, radius=self.vision)
        lowest_price_seller = None
        lowest_price = float('inf')

        # 找出视野范围内价格最低的卖家 Find the seller with the lowest price within the field of view
        for cell in neighborhood:
            sellers = [agent for agent in self.model.grid.get_cell_list_contents(cell) if isinstance(agent, Seller)]
            for seller in sellers:
                if seller.price < lowest_price:
                    lowest_price = seller.price
                    lowest_price_seller = seller

        if lowest_price_seller and self.money >= lowest_price_seller.price and lowest_price_seller.sugar >= 1:
            # 计算最大可交易量 # Calculate the maximum tradable volume
            max_trade_amount = min(self.money // lowest_price_seller.price, lowest_price_seller.sugar)
            trade_amount = math.ceil(max_trade_amount / 2)

            # 进行交易 # Trading
            self.money -= trade_amount * lowest_price_seller.price
            self.sugar += trade_amount
            lowest_price_seller.money += trade_amount * lowest_price_seller.price
            lowest_price_seller.sugar -= trade_amount
                
    def metabolize(self):
        self.sugar -= self.metabolism
        if self.sugar <= 0:
            self.model.grid.remove_agent(self)
            self.model.schedule.remove(self)

class Seller(Agent):
    def __init__(self, unique_id, model, pos, sugar, initial_money):
        super().__init__(unique_id, model)
        self.pos = pos
        self.sugar = sugar
        self.money = initial_money
        self.metabolism = GLOBAL_METABOLISM
        self.vision = GLOBAL_VISION
        self.price = 1

    def step(self):
        self.move()
        self.collect_sugar()
        self.metabolize()
        self.set_price()
        
    def move(self):
        neighborhood = self.model.grid.get_neighborhood(self.pos, moore=True, radius=self.vision)
        valid_cells = [cell for cell in neighborhood if any(isinstance(obj, SugarPatch) for obj in self.model.grid.get_cell_list_contents(cell))]

        if not valid_cells:
            return  # No available moves, stay in place

        max_sugar = max(max(obj.sugar for obj in self.model.grid.get_cell_list_contents(pos) if isinstance(obj, SugarPatch)) for pos in valid_cells)
        max_sugar_cells = [pos for pos in valid_cells if any(obj.sugar == max_sugar for obj in self.model.grid.get_cell_list_contents(pos) if isinstance(obj, SugarPatch))]
        new_position = self.random.choice(max_sugar_cells)

        self.model.grid.move_agent(self, new_position)

    def collect_sugar(self):
        patch = next((obj for obj in self.model.grid.get_cell_list_contents([self.pos]) if isinstance(obj, SugarPatch)), None)
        if patch:
            self.sugar += patch.sugar
            patch.sugar = 0

    def set_price(self):
        if self.money > 2 * GLOBAL_THRESHOLD:
            self.price = 1 / 2
        elif self.money > GLOBAL_THRESHOLD:
            self.price = 1
        else:
            self.price = 2 

    def metabolize(self):
        self.money -= self.metabolism
        if self.money <= 0:
            self.model.grid.remove_agent(self)
            self.model.schedule.remove(self)


# 定义糖地块类 # Define the sugar plot class
class SugarPatch(Agent):
    def __init__(self, unique_id, model, pos, max_sugar):
        super().__init__(unique_id, model)
        self.pos = pos
        self.max_sugar = max_sugar
        self.sugar = random.randint(0, max_sugar)
    
    def step(self):
        self.regrow()
        
    def regrow(self):
        if self.sugar < self.max_sugar:
            self.sugar += random.randint(0, 2 * GLOBAL_REGROW_RATE)

# 定义金钱地块类 # Define the money plot class
class MoneyPatch(Agent):
    def __init__(self, unique_id, model, pos, max_money):
        super().__init__(unique_id, model)
        self.pos = pos
        self.max_money = max_money
        self.money = random.randint(0, max_money)
    
    def step(self):
        self.regrow()
        
    def regrow(self):
        if self.money < self.max_money:
            self.money += random.randint(0, 2 * GLOBAL_REGROW_RATE)


# 定义Sugarscape模型 # Define the Sugarscape model
class Sugarscape(Model):
    def __init__(self, width, height, initial_buyers, initial_sellers):
        super().__init__()
        self.grid = MultiGrid(width, height, torus=False)
        self.schedule = RandomActivation(self)
        self.datacollector = DataCollector(
            {
                "BuyerCount": lambda m: sum(1 for a in m.schedule.agents if isinstance(a, Buyer)),
                "SellerCount": lambda m: sum(1 for a in m.schedule.agents if isinstance(a, Seller)),
                "TotalMoney": lambda m: sum(a.money for a in m.schedule.agents if isinstance(a, (Buyer, Seller))),
            }
        )

        for (contents, pos) in self.grid.coord_iter():
            if random.random() < 0.5:
                max_sugar = GLOBAL_MAX_SUGAR
                patch = SugarPatch(self.next_id(), self, pos, max_sugar)
            else:
                max_money = GLOBAL_MAX_SUGAR  # 假设金钱的最大值与糖相同 
                # Assume the maximum value of money is the same as that of sugar
                patch = MoneyPatch(self.next_id(), self, pos, max_money)
            with warnings.catch_warnings():
                warnings.simplefilter("ignore", UserWarning)
                self.grid.place_agent(patch, pos)
            self.schedule.add(patch)

        for i in range(initial_buyers):
            x, y = self.random.randrange(self.grid.width), self.random.randrange(self.grid.height)
            money = GLOBAL_INIT_AGENT_SUGAR  
            initial_sugar = GLOBAL_INIT_AGENT_SUGAR
            buyer = Buyer(self.next_id(), self, (x, y), money, initial_sugar)
            with warnings.catch_warnings():
                warnings.simplefilter("ignore", UserWarning)
                self.grid.place_agent(buyer, (x, y))
            self.schedule.add(buyer)

        for i in range(initial_sellers):
            x, y = self.random.randrange(self.grid.width), self.random.randrange(self.grid.height)
            sugar = GLOBAL_INIT_AGENT_SUGAR
            initial_money = GLOBAL_INIT_AGENT_SUGAR  
            seller = Seller(self.next_id(), self, (x, y), sugar, initial_money)
            with warnings.catch_warnings():
                warnings.simplefilter("ignore", UserWarning)
                self.grid.place_agent(seller, (x, y))
            self.schedule.add(seller)

        
    def step(self):
        self.schedule.step()
        self.datacollector.collect(self)


# 代理可视化 Agent Visualization
def agent_portrayal(agent):
    if isinstance(agent, Buyer):
        portrayal = {
            "Shape": "circle",
            "Filled": "true",
            "r": 0.5,
            "Color": "blue",
            "Layer": 1,
            "text": agent.money,
            "text_color": "white",
        }
    elif isinstance(agent, Seller):
        portrayal = {
            "Shape": "circle",
            "Filled": "true",
            "r": 0.5,
            "Color": "red",
            "Layer": 1,
            "text": agent.sugar,
            "text_color": "white",
        }
    elif isinstance(agent, SugarPatch):
        portrayal = {
            "Shape": "rect",
            "w": 1,
            "h": 1,
            "Filled": "true",
            "Color": "green",
            "Layer": 0,
            "text": agent.sugar,
            "text_color": "black",
        }
    elif isinstance(agent, MoneyPatch):
        portrayal = {
            "Shape": "rect",
            "w": 1,
            "h": 1,
            "Filled": "true",
            "Color": "yellow",
            "Layer": 0,
            "text": agent.money,
            "text_color": "black",
        }
    return portrayal


grid = CanvasGrid(agent_portrayal, 50, 50, 1500, 1500)

# Display the TotalMoney chart alone
money_chart = ChartModule(
    [
        {"Label": "TotalMoney", "Color": "Black"},
    ],
    data_collector_name='datacollector'
)

# Display the chart of BuyerCount and SellerCount
agent_count_chart = ChartModule(
    [
        {"Label": "BuyerCount", "Color": "Blue"},
        {"Label": "SellerCount", "Color": "Red"},
    ],
    data_collector_name='datacollector'
)

## Count Parameter rationality

In [ ]:
"""
GLOBAL_METABOLISM = 3
GLOBAL_REGROW_RATE = 1
GLOBAL_VISION = 2
GLOBAL_MAX_SUGAR = 5
GLOBAL_MAX_MONEY = 5
GLOBAL_INIT_AGENT_SUGAR = 20
GLOBAL_INIT_AGENT_MONEY = 20
GLOBAL_THRESHOLD = 10
"""

def Parameter_rationality():
    return GLOBAL_METABOLISM / GLOBAL_MAX_SUGAR
print(Parameter_rationality())

## Run

In [ ]:
server = ModularServer(Sugarscape,
                       [grid, money_chart, agent_count_chart],
                       "Sugarscape Model",
                       {"width": 50, "height": 50, "initial_buyers": 50, "initial_sellers": 50})

server.port = 8549
server.launch()
